In [1]:
import numpy as np
import pandas as pd
import wfdb
import ast
import glob
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
import os

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import math
import tensorflow as tf
from tensorflow.keras import initializers, regularizers, constraints
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,normalize, MinMaxScaler
from scipy.signal import spectrogram, resample
from matplotlib.collections import LineCollection

import os
import wandb
from sklearn.metrics import roc_auc_score, classification_report, accuracy_score
from wandb.keras import WandbCallback
import warnings

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.autograd import Variable 
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.nn.parameter import Parameter

import random
from tqdm import tqdm
import wandb
from sklearn.metrics import cohen_kappa_score, accuracy_score,f1_score
from sklearn.utils import shuffle

In [3]:
### Preprocessing 
#   Using the super classes, multi label classification, excluding samples with no labels and considering atleast one label

path = 'C:/ptb/'
Y = pd.read_csv(path+ 'ptbxl_database.csv', index_col = 'ecg_id')



data = np.array([wfdb.rdsamp(path+f)[0] for f in Y.filename_lr])
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))
    
agg_df = pd.read_csv(path+ 'scp_statements.csv', index_col = 0)

agg_df = agg_df[agg_df.diagnostic == 1]

def agg(y_dic):
    temp =[]
    
    for key in y_dic.keys():
        if key in agg_df.index:
            c = agg_df.loc[key].diagnostic_class
            if str(c) != 'nan':
                temp.append(c)
    return list(set(temp))

Y['diagnostic_superclass'] = Y.scp_codes.apply(agg)
Y['superdiagnostic_len'] = Y['diagnostic_superclass'].apply(lambda x: len(x))


#########

counts = pd.Series(np.concatenate(Y.diagnostic_superclass.values)).value_counts()

Y['diagnostic_superclass'] = Y['diagnostic_superclass'].apply(lambda x: list(set(x).intersection(set(counts.index.values))))

X_data = data[Y['superdiagnostic_len'] >= 1]
Y_data = Y[Y['superdiagnostic_len'] >= 1]

mlb = MultiLabelBinarizer()
mlb.fit(Y_data['diagnostic_superclass'])
y = mlb.transform(Y_data['diagnostic_superclass'].values)

########

## Stratify split

X_train = X_data[Y_data.strat_fold < 9]
y_train = y[Y_data.strat_fold < 9]

X_val = X_data[Y_data.strat_fold == 9]
y_val = y[Y_data.strat_fold == 9]

X_test = X_data[Y_data.strat_fold == 10]
y_test = y[Y_data.strat_fold == 10]

del X_data, Y_data, y

In [4]:
# Standardizing

def apply_scaler(X, scaler):
    X_tmp = []
    for x in X:
        x_shape = x.shape
        X_tmp.append(scaler.transform(x.flatten()[:,np.newaxis]).reshape(x_shape))
    X_tmp = np.array(X_tmp)
    return X_tmp


scaler = StandardScaler()

scaler.fit(np.vstack(X_train).flatten()[:,np.newaxis].astype(float))

X_train_scale = apply_scaler(X_train, scaler)
X_test_scale = apply_scaler(X_test, scaler)
X_val_scale = apply_scaler(X_val, scaler)

del X_train, X_test, X_val


In [5]:
class DataGen(Dataset):
    def __init__(self, X, y,batch_size = 16):
        self.batch_size = batch_size
        self.X = X
        self.y = y
        
    def __len__(self):
        return math.ceil(len(self.X) / self.batch_size)
    
    def __getitem__(self,idx):
        
        batch_x = self.X[idx * self.batch_size:(idx + 1) *self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) *self.batch_size]
        
        return torch.tensor(batch_x, dtype = torch.float32), torch.tensor(batch_y, dtype = torch.float32)
    
## Params

batch_size = 16
    
train_gen = DataGen(X_train_scale, y_train, batch_size = batch_size)
test_gen = DataGen(X_test_scale, y_test, batch_size = batch_size)

In [6]:
class ResBlock(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, downsample=None):
        super(ResBlock, self).__init__()
        self.bn1 = nn.BatchNorm1d(num_features=in_channels)
        self.relu = nn.ReLU(inplace=False)
        self.dropout = nn.Dropout(p=0.1, inplace=False)
        self.conv1 = nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size,
                               stride=stride, padding=padding, bias=False)
        self.bn2 = nn.BatchNorm1d(num_features=out_channels)
        self.conv2 = nn.Conv1d(in_channels=out_channels, out_channels=out_channels, kernel_size=kernel_size,
                               stride=stride, padding=padding, bias=False)
        self.maxpool = nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
        self.downsample = downsample




    def forward(self, x):
        identity = x

        out = self.bn1(x)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.conv1(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.conv2(out)

        if self.downsample is not None:
            out = self.maxpool(out)
            identity = self.downsample(x)

        out += identity
        # print(out.shape)

        return out




class ECGNet(nn.Module):

    def __init__(self, struct=[15, 17, 19, 21], in_channels=12, fixed_kernel_size=17, num_classes=5):
        super(ECGNet, self).__init__()
        self.struct = struct
        self.planes = 16
        self.parallel_conv = nn.ModuleList()

        for i, kernel_size in enumerate(struct):
            sep_conv = nn.Conv1d(in_channels=in_channels, out_channels=self.planes, kernel_size=kernel_size,
                               stride=1, padding=0, bias=False)
            self.parallel_conv.append(sep_conv)
        # self.parallel_conv.append(nn.Sequential(
        #     nn.MaxPool1d(kernel_size=2, stride=2, padding=0),
        #     nn.Conv1d(in_channels=1, out_channels=self.planes, kernel_size=1,
        #                        stride=1, padding=0, bias=False)
        # ))

        self.bn1 = nn.BatchNorm1d(num_features=self.planes)
        self.relu = nn.ReLU(inplace=False)
        self.conv1 = nn.Conv1d(in_channels=self.planes, out_channels=self.planes, kernel_size=fixed_kernel_size,
                               stride=2, padding=2, bias=False)
        self.block = self._make_layer(kernel_size=fixed_kernel_size, stride=1, padding=8)
        self.bn2 = nn.BatchNorm1d(num_features=self.planes)
        self.avgpool = nn.AvgPool1d(kernel_size=8, stride=8, padding=2)
        self.rnn = nn.LSTM(input_size=12, hidden_size=40, num_layers=1, bidirectional=False)
        self.fc = nn.Linear(in_features=168, out_features=num_classes)


    def _make_layer(self, kernel_size, stride, blocks=15, padding=0):
        layers = []
        downsample = None
        base_width = self.planes

        for i in range(blocks):
            if (i + 1) % 4 == 0:
                downsample = nn.Sequential(
                    nn.Conv1d(in_channels=self.planes, out_channels=self.planes + base_width, kernel_size=1,
                               stride=1, padding=0, bias=False),
                    nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
                )
                layers.append(ResBlock(in_channels=self.planes, out_channels=self.planes + base_width, kernel_size=kernel_size,
                                       stride=stride, padding=padding, downsample=downsample))
                self.planes += base_width
            elif (i + 1) % 2 == 0:
                downsample = nn.Sequential(
                    nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
                )
                layers.append(ResBlock(in_channels=self.planes, out_channels=self.planes, kernel_size=kernel_size,
                                       stride=stride, padding=padding, downsample=downsample))
            else:
                downsample = None
                layers.append(ResBlock(in_channels=self.planes, out_channels=self.planes, kernel_size=kernel_size,
                                       stride=stride, padding=padding, downsample=downsample))

        return nn.Sequential(*layers)



    def forward(self, x):
        out_sep = []

        for i in range(len(self.struct)):
            sep = self.parallel_conv[i](x)
            out_sep.append(sep)

        out = torch.cat(out_sep, dim=2)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv1(out)  # out => [b, 16, 9960]

        out = self.block(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.avgpool(out)  # out => [b, 64, 10]
        out = out.reshape(out.shape[0], -1)  # out => [b, 640]

        rnn_out, (rnn_h, rnn_c) = self.rnn(x.permute(2, 0, 1))
        new_rnn_h = rnn_h[-1, :, :]  # rnn_h => [b, 40]

        new_out = torch.cat([out, new_rnn_h], dim=1)  # out => [b, 680]
        
        result = self.fc(new_out)  # out => [b, 20]

        # print(out.shape)

        return result

model = ECGNet()

In [6]:
if not os.path.exists('ECGNet_saves'):
    os.mkdir('ECGNet_saves')

In [7]:
def metrics(y_true, y_scores):
    y_pred = y_scores >= 0.5
    acc = np.zeros(y_pred.shape[-1])
    
    roc_auc = roc_auc_score(y_true, y_scores, average = 'macro')
    
    for i in range(y_pred.shape[-1]):
        acc[i] = accuracy_score(y_true[:,i], y_pred[:,i])
    return acc, np.mean(acc), roc_auc

wandb.init(project = 'BaseECG', name = 'ECGNet')

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: likith012 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.22 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [8]:
def train_model(model, optimizer, loss_func, dataset, epoch):

    model.train()
    
    pred_all = []
    loss_all = []
    gt_all = []
    
    for batch_step in tqdm(range(len(dataset)) , desc="train"):
        batch_x, batch_y = dataset[batch_step]    
        batch_x = batch_x.cuda()
        batch_x = batch_x.permute(0,2,1)
        batch_y = batch_y.cuda()

        pred = model(batch_x)
        pred_all.append(pred.cpu().detach().numpy())
        
        loss = loss_func(pred, batch_y)
        loss_all.append(loss.cpu().detach().item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        gt_all.extend(batch_y.cpu().detach().numpy())

    print('epoch {0} '.format(epoch))
    print('train_loss ', np.mean(loss_all))
    pred_all = np.concatenate(pred_all, axis=0)

    _, mean_acc, roc_score = metrics(np.array(gt_all), pred_all )
    wandb.log({'train_mean_accuracy' : mean_acc, 'epoch':epoch})
    wandb.log({'train_roc_score' : roc_score, 'epoch':epoch})
    wandb.log({'train_loss' : np.mean(loss_all) , 'epoch':epoch})

    return np.mean(loss_all)



def test_model(model, loss_func, dataset):

    model.eval()
    
    pred_all = []
    loss_all = []
    gt_all = []
    
    for batch_step in tqdm(range(len(dataset)) , desc="test"):
        batch_x, batch_y = dataset[batch_step]
        batch_x = batch_x.cuda()
        batch_x = batch_x.permute(0,2,1)
        batch_y = batch_y.cuda()
        
        pred = model(batch_x)
        pred_all.append(pred.cpu().detach().numpy())
       
        loss = loss_func(pred, batch_y)
        loss_all.append(loss.cpu().detach().numpy())
        gt_all.extend(batch_y.cpu().detach().numpy())

    print('test_loss ', np.mean(loss_all))
    pred_all = np.concatenate(pred_all, axis=0)

    _, mean_acc, roc_score = metrics(np.array(gt_all), pred_all )
    wandb.log({'test_mean_accuracy' : mean_acc, 'epoch':epoch})
    wandb.log({'test_roc_score' : roc_score, 'epoch':epoch})
    wandb.log({'test_loss' : np.mean(loss_all) , 'epoch':epoch})

    return np.mean(loss_all), mean_acc, roc_score

In [ ]:
lr = 0.001
epochs = 60

model.cuda()
# wandb.watch(model)

optimizer = optim.Adam(model.parameters(), lr = lr)
loss_func = torch.nn.BCEWithLogitsLoss()


for epoch in range(epochs):
    train_step = train_model(model, optimizer, loss_func, train_gen, epoch)
    test_step = test_model(model, loss_func, test_gen)

    if epoch > 5 :
        torch.save(model.state_dict(), f'ECGNet_saves/{epoch}__{test_step[2]:.4f}.pt')


test:   3%|▎         | 4/136 [00:00<00:03, 37.35it/s]

epoch 0 
train_loss  0.36884335230722604


train:   0%|          | 1/1070 [00:00<01:50,  9.66it/s]

test_loss  0.359931


test:   2%|▏         | 3/136 [00:00<00:04, 27.92it/s]

...

test:   3%|▎         | 4/136 [00:00<00:03, 39.94it/s]

epoch 41 
train_loss  0.057202074084121944


test: 100%|██████████| 136/136 [00:03<00:00, 40.42it/s]


test_loss  0.7991146


train:  52%|█████▏    | 558/1070 [00:45<00:40, 12.77it/s]

In [7]:
PATH = r'C:\Users\likit\OneDrive\Desktop\Cardio-Viz\Code\BaseECG\ECGNet\11__0.9102.pt'

model.load_state_dict(torch.load(PATH))
model.eval()

ECGNet(
  (parallel_conv): ModuleList(
    (0): Conv1d(12, 16, kernel_size=(15,), stride=(1,), bias=False)
    (1): Conv1d(12, 16, kernel_size=(17,), stride=(1,), bias=False)
    (2): Conv1d(12, 16, kernel_size=(19,), stride=(1,), bias=False)
    (3): Conv1d(12, 16, kernel_size=(21,), stride=(1,), bias=False)
  )
  (bn1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (conv1): Conv1d(16, 16, kernel_size=(17,), stride=(2,), padding=(2,), bias=False)
  (block): Sequential(
    (0): ResBlock(
      (bn1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (dropout): Dropout(p=0.1, inplace=False)
      (conv1): Conv1d(16, 16, kernel_size=(17,), stride=(1,), padding=(8,), bias=False)
      (bn2): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(16, 16, kernel_size=(17,), stride=(1,), padding=(8,), bias=False)
      (maxpool): MaxPoo

In [8]:
test_gen = DataGen(X_test_scale, y_test, batch_size=len(X_test_scale))

batch = test_gen[0][0].permute(0,2,1)
pred = model(batch).detach()

roc_auc_score(y_test, pred, average='macro')

0.9101563184785769

In [9]:
### Class wise AUC

roc_score = roc_auc_score(y_test, pred, average='macro')
print(f'roc_score : {roc_score}')

def AUC(y_true: np.ndarray, y_pred: np.ndarray, verbose=False) -> float:
    """Computes the macro-average AUC score.

    Args:
        y_true (np.ndarray): list of labels
        y_pred (np.ndarray): list of predicted probabilities

    Returns:
        float: macro-average AUC score.
    """
    aucs = []
    assert len(y_true.shape) == 2 and len(y_pred.shape) == 2, 'Predictions and labels must be 2D.'
    for col in range(y_true.shape[1]):
        try:
            aucs.append(roc_auc_score(y_true[:, col], y_pred[:, col]))
        except ValueError as e:
            if verbose:
                print(
                    f'Value error encountered for label {col}, likely due to using mixup or '
                    f'lack of full label presence. Setting AUC to accuracy. '
                    f'Original error was: {str(e)}.'
                )
            aucs.append((y_pred == y_true).sum() / len(y_pred))
    return np.array(aucs)

class_auc = AUC(y_test, pred)
print(f'class wise AUC : {class_auc}')



roc_score : 0.9101563184785769
class wise AUC : [0.91022348 0.88426456 0.9021273  0.93360477 0.92056149]


In [10]:
### Accuracy metric

def metrics(y_true, y_scores):
    y_pred = y_scores >= 0.5
    acc = np.zeros(y_pred.shape[-1])
    
    for i in range(y_pred.shape[-1]):
        acc[i] = accuracy_score(y_true[:,i], y_pred[:,i])
    return acc, np.mean(acc)

acc, mean_acc = metrics(y_test, pred)
print(f'class wise accuracy: {acc}')
print(f'accuracy: {mean_acc}')

class wise accuracy: [0.88719371 0.90661119 0.84604716 0.85899214 0.86870088]
accuracy: 0.8735090152565881


In [11]:
pred_values = pred >= 0.5

report = classification_report(y_test, pred_values, target_names = mlb.classes_)
print(report)

              precision    recall  f1-score   support

          CD       0.86      0.61      0.71       498
         HYP       0.77      0.33      0.47       263
          MI       0.80      0.53      0.64       553
        NORM       0.79      0.92      0.85       964
        STTC       0.83      0.58      0.68       523

   micro avg       0.81      0.67      0.73      2801
   macro avg       0.81      0.59      0.67      2801
weighted avg       0.81      0.67      0.72      2801
 samples avg       0.75      0.70      0.71      2801

C:\Users\likit\anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
def multi_threshold_precision_recall(y_true: np.ndarray, y_pred: np.ndarray, thresholds: np.ndarray) :
    
    # Expand analysis to number of thresholds
    y_pred_bin = np.repeat(y_pred[None, :, :], len(thresholds), axis=0) >= thresholds[:, None, None]
    y_true_bin = np.repeat(y_true[None, :, :], len(thresholds), axis=0)

    # Compute true positives
    TP = np.sum(np.logical_and(y_true, y_pred_bin), axis=2)

    # Compute macro-average precision handling all warnings
    with np.errstate(divide='ignore', invalid='ignore'):
        den = np.sum(y_pred_bin, axis=2)
        precision = TP / den
        precision[den == 0] = np.nan
        with warnings.catch_warnings():  # for nan slices
            warnings.simplefilter("ignore", category=RuntimeWarning)
            av_precision = np.nanmean(precision, axis=1)

    # Compute macro-average recall
    recall = TP / np.sum(y_true_bin, axis=2)
    av_recall = np.mean(recall, axis=1)

    return av_precision, av_recall


def metric_summary(y_true: np.ndarray, y_pred: np.ndarray, num_thresholds: int = 10) :
    
    thresholds = np.arange(0.00, 1.01, 1. / (num_thresholds - 1), float)
    average_precisions, average_recalls = multi_threshold_precision_recall(
        y_true, y_pred, thresholds
    )
    f_scores = 2 * (average_precisions * average_recalls) / (average_precisions + average_recalls)
    auc = np.array(AUC(y_true, y_pred, verbose=True)).mean()
    return (
        f_scores[np.nanargmax(f_scores)],
        auc,
        f_scores,
        average_precisions,
        average_recalls,
        thresholds
    )

metric_summary(y_test, pred.numpy())


(0.7712464813278368,
 0.9101563184785769,
 array([0.77124648, 0.76814814, 0.76475761, 0.76157367, 0.75672646,
        0.75207953, 0.74914798, 0.74659073, 0.7404506 , 0.73561017]),
 array([0.7926559 , 0.79706679, 0.8023798 , 0.80757946, 0.81168563,
        0.81620603, 0.82248596, 0.82982319, 0.83320099, 0.8367813 ]),
 array([0.75096317, 0.74125443, 0.73050547, 0.72052705, 0.70873786,
        0.69729542, 0.68781785, 0.6785329 , 0.6662814 , 0.65626445]),
 array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
        0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ]))